In [1]:
#Archive afkortingen namen
Archief_afk_onderdelen = {'NA': 'Notarieel Archief', 'ONA': 'Oud Notarieel Archief', 'PA': 'Particulier Archief'}
Archief_afk_namen = {'SA': 'Stadsarchief Amsterdam', 'GAL': 'Gemeente Archief Leiden', 'HNA': 'Het Nationaal Archief', 'GAR': 'Gemeente Archief Rotterdam','HUA': 'Het Utrechts Archief'}

In [1]:
import os
from lxml import etree 

In [2]:
doc = etree.parse('../Data/xml_data/provenance.xml')
provenance = doc.findall('recordList/record/provenance/provenance.project.txt')
for item in provenance:
    if item.text == None:
        provenance.remove(item)
    else:
        continue

In [3]:
estate_inventory_items = list()
for elements in provenance:
    element = elements.text
    element_list = element.split(';')
    for elem in element_list:
        if 'Estate inventory' in elem:
            estate_inventory_items.append(elem)
        if 'estate inventory' in elem:
            estate_inventory_items.append(elem)

In [7]:
for item in estate_inventory_items[0:21]:
    print(item)

{HRvA, Familiearchief Van Slingelandt, inv. no. 19, p. 9, no. 34.} ? their estate inventory, 1761
 ? estate inventory, Anna van Renesse, Lady of Assendelft (1622-67), Kasteel Assumburgh, near Heemstede, 1667, ‘Old gentleman’s room above the great chamber’ (‘een oud schilderije’)
 ? estate inventory Jan Six (1618-1700) and Margareta Tulp (1634-1709), Herengracht, Amsterdam, 6 January 1710, in the art room (‘Een avondtstondt van Aart van der Neer met beelden etc. [fl] 15’)
 ? estate inventory, Clara de Valaer (1584-1660), widow successfully of Eduart van Domselaer (1568-1624) and Hendrick van Domselaer (1580/81-1652), Amsterdam, 16 October 1660, as security (‘een stuk synde twee pauwen ende een kint van Rembrant’)
 Strauss et al. 1979, p. 461, doc. no. 1660/15 } ? estate inventory of her son, Tobias van Domselaer (1611-85), Amsterdam, September 1685 (‘een groot schilderij met twee paeuwen van Rembrandt’)
? Estate inventory, Marten Davidsz van Ceulen (1617-87), Oudezijds Voorburgwal [no. 

In [9]:
#code just for the items mentioning estate inventory

for item in estate_inventory_items:
    #possible parts of the provenance sentence
    name = ''
    name2 = '' # if there's a second owner
    BDdate = '' #birth-death date
    BDdate2 = ''
    extra = ''
    extra2 = ''
    place = ''
    date = '' #date of inventory
    location = '' #location of work in house
    no = '' #no in inventory
    attribution = ''
    description = ''
    source = ''
    start_source = ''
    #fix possible missing comma at the start
    if 'estate inventory ' in item:
        item = item.replace('estate inventory', 'estate inventory,')
    #saving the original, but fixed, item in case I need it
    start_item = item
    #certain or uncertain provenance
    certain_uncertain = item.split('(')
    if '?' in certain_uncertain[0]:
        #provenance data is uncertain
        proper_format = '?'
        readable_sent = 'Possibly part of'
    else:
        #provenance data is certain
        proper_format = ''
        readable_sent = 'Part of'
    
    #source
    #if item starts with source
    if item.startswith('{') or item.startswith(' {'):
        curly_split = item.split('}')
        source = curly_split[0].replace('{', '').strip()
        start_source = '{'+source+'}'
    #if item ends with source
    if '{' in item:
        curly_split = item.split('{')
        source = curly_split[1].replace('}', '').strip()
        source = '{'+source+'}'
    sourceless_item = start_item.replace(source,'').strip()
    
    #name owner
    comma_split = sourceless_item.split(',')
    #no owner name, owner name in previous provenance sentence
    if 'his ' in comma_split[0]:
        proper_format = f"{proper_format} his"
        readable_sent = f"{readable_sent} his"
    elif 'her ' in comma_split[0]:
        proper_format = f"{proper_format} her"
        readable_sent = f"{readable_sent} her"
    elif 'their ' in comma_split[0]:
        proper_format = f"{proper_format} their"
        readable_sent = f"{readable_sent} their"
    #owner name is mentioned
    else:
        name_elem = comma_split[1]
        if 'and' in name_elem:
            two_names = name_elem.split('and')
            name_one = two_names[0]
            name = name_one.strip()
            if '(' in name_one:
                name_one = name_one.split('(')
                name = name_one[0].strip()
                #BDdate person 1
                BDdate = name_one[1].replace(')','').strip()
            name_two = two_names[1]
            name2 = name_two.strip()
            if '(' in name_two:
                name_two = name_two.split('(')
                name2 = name_two[0].strip
                #BDdate person 2
                BDdate2 = name_two[1].replace(')','').strip()
                
    #inventory date
    for item in comma_split:
        if '1' in item or '2' in item and not '(' in item:
            date = item
        #no in inventory
        if 'no.' in item:
            no = item
            if '(' in item:
                split_item = item.split('(')
                no = split_item[0]
            
    #description
    para_split = sourceless_item.split('(')
    for item in para_split:
        if "')" in item:
            item = item.split(')')
            description = item[1].strip()   